# Capstone Notebook

This Notebook will be used to work through the Capstone course. 

One project will be to find the ideal neighborhood to move to for your new job. 
You just got a new job and it's too far away, so you need to move. You would like to move
to a neighborhood that is close to your new job and also similar to your current neighborhood

In [ ]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

In [ ]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

df.rename(columns={'Postcode': 'PostalCode'}, inplace=True)


In [ ]:
# filter out unassigned boroughs
filtered_data = df[df['Borough' ] != 'Not assigned'].reset_index(drop=True)
   
filtered_data


In [115]:
#test = filtered_data[filtered_data['PostalCode'] == 'M6A']


# group by postal code to show neighborhoods sharing same postal code
groups = filtered_data.groupby(['PostalCode', 'Borough']).agg(lambda x: ','.join(x))


#groups.groups

groups
#filtered_data.groupby(['PostalCode']).value_counts()




,,Neighbourhood
PostalCode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
